In [ ]:
cd

In [ ]:
project_path = "/home/jupyter"

import sys

sys.path.append(project_path)
import re

import numpy as np
import pandas as pd
from google.cloud import bigquery

from fintrans_toolbox.src.utils import bq_utils as bq
from fintrans_toolbox.src.utils import table_utils as t

In [ ]:
client = bigquery.Client()

In [ ]:
# Merchant locations from retail highstreets
sql_merchant_loc = f"SELECT DISTINCT merchant_location FROM `ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns_dedupe` WHERE merchant_location_level = 'POSTAL_SECTOR' and mcg = 'All'"
df_merchant = bq.read_bq_table_sql(client, sql_merchant_loc)

# Cardholder locations from retail highstreets
sql_cardholder_loc = f"SELECT DISTINCT cardholder_location FROM `ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns_dedupe` WHERE cardholder_location_level = 'POSTAL_SECTOR' and mcg = 'All'"
df_cardholder = bq.read_bq_table_sql(client, sql_cardholder_loc)

# Merchant locations from spend merchant location
sql = f"SELECT DISTINCT merchant_location FROM `ons-fintrans-data-prod.fintrans_visa.spend_merchant_location_dedupe` WHERE merchant_location_level = 'POSTAL_SECTOR' and mcg = 'All'"
df_sml = bq.read_bq_table_sql(client, sql)

# cardholder locations from retail highstreets
sql = f"SELECT DISTINCT cardholder_location FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel_dedupe` WHERE cardholder_origin_country = 'POSTAL_SECTOR' and mcg = 'All'"
df_spoc = bq.read_bq_table_sql(client, sql)

In [ ]:
df_merchant = df_merchant.rename(columns={"merchant_location": "cardholder_location"})
df_sml = df_sml.rename(columns={"merchant_location": "cardholder_location"})
df = pd.concat([df_merchant, df_cardholder, df_spoc, df_sml])
df = df.drop_duplicates().reset_index()

In [ ]:
df

In [ ]:
df["postcode_district"] = df["cardholder_location"].str[:-2]

In [ ]:
df

In [ ]:
df["postcode_area"] = (
    df["postcode_district"].str.findall("([a-zA-Z ]*)\d*.*").apply("".join)
)
df